In [54]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [55]:
df=pd.read_csv('train_data.csv')


In [187]:
X=np.array(df.drop(['y','datetime'],axis=1).values)
y=np.array(df['y'].values)
X=X.reshape(X.shape[0],X.shape[1],1)
X_train,X_test,y_train,y_test=train_test_split(X,y,shuffle=False)
X_train.shape

(1575, 11, 1)

In [167]:
model_save_path = './pose_test.h5'

In [206]:
length_of_sequence = X_train.shape[1] 
in_out_neurons = X_train.shape[0]
n_hidden = 5

model = Sequential()
model.add(LSTM(64, return_sequences=False, activation='relu', input_shape= (11,1)))
model.add(Dense(1))

model.summary()
#model.weights

Model: "sequential_66"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_135 (LSTM)              (None, 64)                16896     
_________________________________________________________________
dense_133 (Dense)            (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [207]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [208]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [209]:
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=20)
# 学習
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/5
11/13 [========================>.....] - ETA: 0s - loss: 3.6169e-06 - categorical_accuracy: 1.0000
Epoch 00001: saving model to ./pose_test.h5
13/13 [==============================] - 1s 48ms/step - loss: 3.6078e-06 - categorical_accuracy: 1.0000 - val_loss: 5.4324e-06 - val_categorical_accuracy: 1.0000
Epoch 2/5
10/13 [======================>.......] - ETA: 0s - loss: 3.6011e-06 - categorical_accuracy: 1.0000
Epoch 00002: saving model to ./pose_test.h5
13/13 [==============================] - 0s 22ms/step - loss: 3.6078e-06 - categorical_accuracy: 1.0000 - val_loss: 5.4324e-06 - val_categorical_accuracy: 1.0000
Epoch 3/5
10/13 [======================>.......] - ETA: 0s - loss: 3.5879e-06 - categorical_accuracy: 1.0000
Epoch 00003: saving model to ./pose_test.h5
13/13 [==============================] - 0s 22ms/step - loss: 3.6078e-06 - categorical_accuracy: 1.0000 - val_loss: 5.4324e-06 - val_categorical_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - ETA:

In [210]:
model.load_weights('./pose_test.h5')
# 評価
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)



loss=mean_squared_error(y_pred,y_test)
loss

2423.3310963933995

In [159]:
import numpy as np
x = np.array([[10,20,30], [20,30,40], [30, 40, 50], [40, 50, 60]])
y = np.array([[40, 50], [50, 60], [60, 70], [70, 80]])
 
# 行列のフォーマット変更。
# LSTMは、入力フォーマットを[サンプルの数, 入力のステップ数(この場合は3), features]とする必要があるためです。
x = x.reshape((x.shape[0], x.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [161]:
x.shape

(4, 3, 1)